In [1]:
import pynq
from pynq import allocate
from pynq.lib.video import *

ol = pynq.Overlay('rasterization9/vitis_design.bit')

ip = ol.rasterization_1

In [2]:
import os
import sys
import cv2
import torch
import torch.nn.functional as F
import numpy as np
import socket
import time

In [3]:
_BASE_DIR = os.getcwd()
sys.path.insert(0, _BASE_DIR)

from utils.rasterization import GaussianRasterizer as Renderer
from utils.rasterization import GaussianRasterizationSettings as Camera
from utils.common_helpers import *
from utils.recon_helpers import *
from utils.slam_helpers import *

In [4]:
from pynq.lib.video import *
displayport = DisplayPort()
print(displayport.modes)
mode = displayport.modes[0]
displayport.configure(VideoMode(1920, 1200, 24), PIXEL_RGB)
displayport.start()

[VideoMode: width=1920 height=1200 bpp=24 fps=60, VideoMode: width=1920 height=1080 bpp=24 fps=60, VideoMode: width=1920 height=1080 bpp=24 fps=60, VideoMode: width=1920 height=1080 bpp=24 fps=60, VideoMode: width=1920 height=1080 bpp=24 fps=50, VideoMode: width=1680 height=1050 bpp=24 fps=60, VideoMode: width=1440 height=900 bpp=24 fps=60, VideoMode: width=1280 height=960 bpp=24 fps=60, VideoMode: width=1280 height=800 bpp=24 fps=60, VideoMode: width=1280 height=720 bpp=24 fps=60, VideoMode: width=1280 height=720 bpp=24 fps=60, VideoMode: width=1280 height=720 bpp=24 fps=50, VideoMode: width=1024 height=768 bpp=24 fps=60, VideoMode: width=800 height=600 bpp=24 fps=60, VideoMode: width=800 height=600 bpp=24 fps=56, VideoMode: width=640 height=480 bpp=24 fps=60, VideoMode: width=640 height=480 bpp=24 fps=60]


In [5]:
def load_camera(cfg, scene_path):
    all_params = dict(np.load(scene_path, allow_pickle=True))
    params = all_params
    org_width = params['width']
    org_height = params['height']
    w2c = torch.eye(4)
    intrinsics = params['intrinsics']
    k = intrinsics[:3, :3]

    # Scale intrinsics to match the visualization resolution
    k[0, :] *= cfg['viz_w'] / org_width
    k[1, :] *= cfg['viz_h'] / org_height
    return w2c, k

def load_scene_data(scene_path, first_frame_w2c):
    # Load Scene Data
    all_params = dict(np.load(scene_path, allow_pickle=True))
    all_params = {k: torch.tensor(all_params[k]).cpu().float() for k in all_params.keys()}
    first_frame_w2c = torch.tensor(first_frame_w2c).cpu().float()

    keys = [k for k in all_params.keys() if
            k in ['means3D', 'colors_precomp', 'opacities', 'scales', 'means2D']]

    params = all_params
    for k in keys:
        if not isinstance(all_params[k], torch.Tensor):
            params[k] = torch.tensor(all_params[k]).cpu().float()
        else:
            params[k] = all_params[k].cpu().float()
    
    opacities = torch.sigmoid(params['logit_opacities'])
    rendervar = {
        'md_1_for': torch.cat([params['means3D'], params["scales"], params['rgb_colors'], opacities ], dim=1).contiguous(),
        'means2D': torch.zeros_like(params['means3D'], requires_grad=True, device="cpu") + 0,
    }
    params['thresholds'] = [0.3, 0.481, 0.556, 0.613, 0.661, 0.704, 0.743, 0.778, 0.811, 0.842, 0.872, 0.899, 0.926, 0.952, 0.976, 1.0, 1.011, 1.023, 1.034, 1.045, 1.057, 1.068, 1.08, 1.091, 1.102, 1.114, 1.125, 1.136, 1.148, 1.159, 1.17, 1.182, 1.193, 1.205, 1.216, 1.227, 1.239, 1.25, 1.261, 1.273, 1.284, 1.295, 1.307, 1.318, 1.33, 1.341, 1.352, 1.364, 1.375, 1.386, 1.398, 1.409, 1.42, 1.432, 1.443, 1.455, 1.466, 1.477, 1.489, 1.5, 1.511, 1.523, 1.534, 1.545, 1.557, 1.568, 1.58, 1.591, 1.602, 1.614, 1.625, 1.636, 1.648, 1.659, 1.67, 1.682, 1.693, 1.705, 1.716, 1.727, 1.739, 1.75, 1.761, 1.773, 1.784, 1.795, 1.807, 1.818, 1.83, 1.841, 1.852, 1.864, 1.875, 1.886, 1.898, 1.909, 1.92, 1.932, 1.943, 1.955, 1.966, 1.977, 1.989, 2.0, 2.011, 2.023, 2.034, 2.045, 2.057, 2.068, 2.08, 2.091, 2.102, 2.114, 2.125, 2.136, 2.148, 2.159, 2.17, 2.182, 2.193, 2.205, 2.216, 2.227, 2.239, 2.25, 2.261, 2.273, 2.284, 2.295, 2.307, 2.318, 2.33, 2.341, 2.352, 2.364, 2.375, 2.386, 2.398, 2.409, 2.42, 2.432, 2.443, 2.455, 2.466, 2.477, 2.489, 2.5, 2.511, 2.523, 2.534, 2.545, 2.557, 2.568, 2.58, 2.591, 2.602, 2.614, 2.625, 2.636, 2.648, 2.659, 2.67, 2.682, 2.693, 2.705, 2.716, 2.727, 2.739, 2.75, 2.761, 2.773, 2.784, 2.795, 2.807, 2.818, 2.83, 2.841, 2.852, 2.864, 2.875, 2.886, 2.898, 2.909, 2.92, 2.932, 2.943, 2.955, 2.966, 2.977, 2.989, 3.0, 3.057, 3.115, 3.174, 3.234, 3.296, 3.358, 3.422, 3.487, 3.553, 3.621, 3.69, 3.76, 3.831, 3.904, 3.978, 4.054, 4.131, 4.209, 4.289, 4.37, 4.453, 4.538, 4.624, 4.712, 4.801, 4.893, 4.986, 5.08, 5.177, 5.275, 5.375, 5.477, 5.581, 5.687, 5.795, 5.905, 6.017, 6.132, 6.248, 6.367, 6.488, 6.611, 6.736, 6.864, 6.995, 7.128, 7.263, 7.401, 7.541, 7.685, 7.831, 7.979, 8.131, 8.285, 8.442, 8.603, 8.766, 8.933, 9.102, 9.275, 9.451, 9.631, 9.814, 9.999]
    return rendervar , params['thresholds']

def render(ip, w2c, k, thresholds, timestep_data, cfg):
    with torch.no_grad():
        cam = setup_camera(cfg['viz_w'], cfg['viz_h'], thresholds, k, w2c, cfg['viz_near'], cfg['viz_far'])
        im, _, _ = Renderer(ip=ip, raster_settings=cam)(**timestep_data)
        return im.detach().numpy()
    
def get_new_cam(data):
    try:
        # 将字节数据解码为字符串
        data_str = data.decode('utf-8').strip()
        print(f"接收到的数据: {data_str}")
        
        # 检查数据格式是否正确
        if data_str.startswith("w2c_matrix:"):
            matrix_str = data_str.replace("w2c_matrix:", "")
            
            float_strs = matrix_str.split(",")
            
            if len(float_strs) == 16:
                # 将字符串转换为浮点数并重塑为4x4矩阵
                float_array = [float(x) for x in float_strs]
                w2c = torch.tensor(float_array).reshape(4, 4).float()
                print("成功解析w2c矩阵:")
                print(w2c)
                return w2c
            else:
                print(f"警告: 矩阵元素数量不正确,期望16个,实际收到{len(float_strs)}个")
        else:
            print(f"警告: 数据格式不正确,期望以'w2c_matrix:'开头")
            
    except Exception as e:
        print(f"解析w2c矩阵时出错: {e}")
    
    # 如果解析失败，返回None，让调用者决定如何处理
    return None

In [6]:
def visualize_sequence(ip, scene_path, cfg, send_sock, recv_sock):
    
    print(f"服务器已启动，监听端口 {server_address[1]}...")

    current_w2c, k = load_camera(cfg, scene_path)
    scene_data, thresholds = load_scene_data(scene_path, current_w2c)
    im = render(ip, current_w2c, k, thresholds, scene_data, cfg)
    
#     im = im.reshape(360, 480, 3) * 255
#     im[im > 255] = 255
#     im[im < 0] = 0
#     img_uint8 = im.astype(np.uint8)
#     img_RGB = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
#     frame = displayport.newframe()
#     np.copyto(frame, cv2.resize(img_RGB, (1920,1200)))
#     displayport.writeframe(frame)

    im = im.reshape(360, 480, 3) * 255
    im[im > 255] = 255
    im[im < 0] = 0
    img_uint8 = im.astype(np.uint8)
    img_RGB = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
    # 获取原始图像尺寸
    orig_height, orig_width = img_RGB.shape[:2]
    orig_height *= 3
    orig_width *= 3
    # 创建黑色背景帧
    img_RGB = cv2.resize(img_RGB, (orig_width,orig_height))
    frame = displayport.newframe()
    frame[:] = 0  # 填充黑色
    # 计算居中位置
    start_x = (1920 - orig_width) // 2
    start_y = (1200 - orig_height) // 2
    # 将图像复制到帧的中央位置
    frame[start_y:start_y+orig_height, start_x:start_x+orig_width] = img_RGB
    displayport.writeframe(frame)

   
    t =time.time()
    while True:
        send_sock.sendto(b'1', ('192.168.0.142', 8777))
        print(f"已向 192.168.5.7 发送确认消息")
        
        try:
            data, address = recv_sock.recvfrom(1024)

            new_w2c = get_new_cam(data)

            im = render(ip, new_w2c, k, thresholds, scene_data, cfg)
            
#             im = im.reshape(360, 480, 3) * 255
#             im[im > 255] = 255
#             im[im < 0] = 0
#             img_uint8 = im.astype(np.uint8)
#             img_RGB = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
#             frame = displayport.newframe()
#             np.copyto(frame, cv2.resize(img_RGB, (1920,1200)))
#             displayport.writeframe(frame)

            im = im.reshape(360, 480, 3) * 255
            im[im > 255] = 255
            im[im < 0] = 0
            img_uint8 = im.astype(np.uint8)
            img_RGB = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
            # 获取原始图像尺寸
            orig_height, orig_width = img_RGB.shape[:2]
            orig_height *= 3
            orig_width *= 3
            img_RGB = cv2.resize(img_RGB, (orig_width,orig_height))
            # 创建黑色背景帧
            frame = displayport.newframe()
            frame[:] = 0  # 填充黑色
            # 计算居中位置
            start_x = (1920 - orig_width) // 2
            start_y = (1200 - orig_height) // 2
            # 将图像复制到帧的中央位置
            frame[start_y:start_y+orig_height, start_x:start_x+orig_width] = img_RGB
            displayport.writeframe(frame)
        
        except socket.timeout:
            # 2秒内没有收到回复，继续下一次循环
            print("等待回复超时，继续下一次发送")
            continue
        except Exception as e:
            # 处理其他可能的异常
           
            print(f"接收数据时发生错误: {e}")
            continue
            
#         if time.time() - t > 5 :break
            

In [7]:
results_dir = os.path.join(_BASE_DIR, "experiments")
scene_path = os.path.join(results_dir, "params.npz")

if not os.path.exists(scene_path):
    print(f"错误: 找不到场景数据文件 {scene_path}")
    sys.exit(1)

# 创建默认配置
class DefaultConfig:
    def __init__(self):
        self.config = {
            "workdir": _BASE_DIR,
            "run_name": "experiments",
            "seed": 42,
            "viz": {
                "viz_w": 640,
                "viz_h": 480,
                "viz_near": 0.01,
                "viz_far": 100,
                "view_scale": 2.0,
                "visualize_cams": False,
                "offset_first_viz_cam": False,
                "render_mode": "centers",
                "show_sil": False
            }
        }

experiment = DefaultConfig()
viz_cfg = experiment.config["viz"]

# 输出目录
output_dir = os.path.join(_BASE_DIR, "rendered_output")

# 创建socket并绑定地址和端口
# 创建两个socket：一个用于发送，一个用于接收
send_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
recv_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# 设置接收socket的超时
recv_sock.settimeout(2)
# 只绑定接收socket到8778端口
server_address = ('0.0.0.0', 8778)
recv_sock.bind(server_address)

# 可视化序列
visualize_sequence(ip, scene_path, viz_cfg, send_sock, recv_sock)

服务器已启动，监听端口 8778...
trying to : pre grouping...


/tmp/ipykernel_9080/174971674.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  first_frame_w2c = torch.tensor(first_frame_w2c).cpu().float()
/home/xilinx/jupyter_notebooks/SGSAM/utils/recon_helpers.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  w2c = torch.tensor(w2c).float()


buffers take : 162.13583946228027 ms    means3D 5.662679672241211 ms    allocate 12.57014274597168 ms    temp_mem_buffer/sortouts_buffer 143.90206336975098 ms
Done! pre_grouping_ip takes : 52.71434783935547 ms
trying to : forward...
buffers take : 177.77585983276367 ms    sum acc_size 152.1279811859131 ms
render ip working time :  122.91121482849121 ms

time pg  273.4043598175049 ms
time fw  303.3618927001953 ms
time ol_forward  594.8340892791748 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
trying to : pre grouping...
buffers take : 101.83477401733398 ms    means3D 3.7119388580322266 ms    allocate 7.042884826660156 ms    temp_mem_buffer/sortouts_buffer 91.07851982116699 ms
Done! pre_grouping_ip takes : 52.54220962524414 ms
trying 

buffers take : 87.76617050170898 ms    means3D 2.6373863220214844 ms    allocate 7.360219955444336 ms    temp_mem_buffer/sortouts_buffer 77.76761054992676 ms
Done! pre_grouping_ip takes : 53.151845932006836 ms
trying to : forward...
buffers take : 174.18575286865234 ms    sum acc_size 148.15139770507812 ms
render ip working time :  164.42155838012695 ms

time pg  160.1402759552002 ms
time fw  341.3078784942627 ms
time ol_forward  519.456148147583 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:1.000000,0.000000,0.000000,0.000000,0.000000,0.819152,-0.573576,0.000000,0.000000,0.573576,0.819152,0.200000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.8192, -0.5736,  0.0000],
        [ 0.0000,  0.5736,  0.8192,  0.2000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
trying to : pre grouping...
buffers take : 91.18366241455078 ms    means3D 5.977153778076172 ms    allocate 7.1773529052734375 ms    temp_mem_buffer/sortouts_buffer

接收到的数据: w2c_matrix:0.984808,0.000000,-0.173648,0.000000,-0.086824,0.866026,-0.492404,0.000000,0.150384,0.500000,0.852869,0.250000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[ 0.9848,  0.0000, -0.1736,  0.0000],
        [-0.0868,  0.8660, -0.4924,  0.0000],
        [ 0.1504,  0.5000,  0.8529,  0.2500],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
trying to : pre grouping...
buffers take : 86.5931510925293 ms    means3D 2.676725387573242 ms    allocate 7.344961166381836 ms    temp_mem_buffer/sortouts_buffer 76.56979560852051 ms
Done! pre_grouping_ip takes : 53.14040184020996 ms
trying to : forward...
buffers take : 177.06537246704102 ms    sum acc_size 149.82891082763672 ms
render ip working time :  180.54914474487305 ms

time pg  158.2334041595459 ms
time fw  360.20922660827637 ms
time ol_forward  536.452054977417 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:0.984808,0.000000,-0.173648,0.000000,-0.086824,0.866026,-0.492404,0.000000,0.150384,0.500000,0.852869,0.250000,0.0000

...

buffers take : 174.84712600708008 ms    sum acc_size 148.75078201293945 ms
render ip working time :  192.5973892211914 ms

time pg  160.21013259887695 ms
time fw  370.147705078125 ms
time ol_forward  548.3639240264893 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:0.995213,0.097735,-0.000735,-0.100000,-0.084959,0.861348,-0.500862,0.000000,-0.048319,0.498526,0.865529,0.400000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[ 9.9521e-01,  9.7735e-02, -7.3500e-04, -1.0000e-01],
        [-8.4959e-02,  8.6135e-01, -5.0086e-01,  0.0000e+00],
        [-4.8319e-02,  4.9853e-01,  8.6553e-01,  4.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
trying to : pre grouping...
buffers take : 87.15605735778809 ms    means3D 2.7167797088623047 ms    allocate 7.259845733642578 ms    temp_mem_buffer/sortouts_buffer 77.1780014038086 ms
Done! pre_grouping_ip takes : 53.12013626098633 ms
trying to : forward...
buffers take : 176.0404109954834 ms    sum acc_size 149.04308319091797

buffers take : 176.1929988861084 ms    sum acc_size 149.80530738830566 ms
render ip working time :  192.57020950317383 ms

time pg  159.8832607269287 ms
time fw  371.45471572875977 ms
time ol_forward  549.3412017822266 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:0.995213,0.097735,-0.000735,-0.100000,-0.084959,0.861348,-0.500862,0.000000,-0.048319,0.498526,0.865529,0.400000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[ 9.9521e-01,  9.7735e-02, -7.3500e-04, -1.0000e-01],
        [-8.4959e-02,  8.6135e-01, -5.0086e-01,  0.0000e+00],
        [-4.8319e-02,  4.9853e-01,  8.6553e-01,  4.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
trying to : pre grouping...
buffers take : 86.38978004455566 ms    means3D 2.726316452026367 ms    allocate 7.1659088134765625 ms    temp_mem_buffer/sortouts_buffer 76.49588584899902 ms
Done! pre_grouping_ip takes : 53.08866500854492 ms
trying to : forward...
buffers take : 176.49579048156738 ms    sum acc_size 149.56235885620

buffers take : 174.6349334716797 ms    sum acc_size 148.60248565673828 ms
render ip working time :  192.5961971282959 ms

time pg  160.66765785217285 ms
time fw  369.9078559875488 ms
time ol_forward  548.5854148864746 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:0.995213,0.097735,-0.000735,-0.100000,-0.084959,0.861348,-0.500862,0.000000,-0.048319,0.498526,0.865529,0.400000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[ 9.9521e-01,  9.7735e-02, -7.3500e-04, -1.0000e-01],
        [-8.4959e-02,  8.6135e-01, -5.0086e-01,  0.0000e+00],
        [-4.8319e-02,  4.9853e-01,  8.6553e-01,  4.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
trying to : pre grouping...
buffers take : 87.36443519592285 ms    means3D 2.663135528564453 ms    allocate 7.11822509765625 ms    temp_mem_buffer/sortouts_buffer 77.58140563964844 ms
Done! pre_grouping_ip takes : 53.27796936035156 ms
trying to : forward...
buffers take : 174.97587203979492 ms    sum acc_size 148.63252639770508

buffers take : 176.04899406433105 ms    sum acc_size 149.07145500183105 ms
render ip working time :  192.5833225250244 ms

time pg  157.7448844909668 ms
time fw  371.4406490325928 ms
time ol_forward  547.0924377441406 ms
已向 192.168.5.7 发送确认消息
接收到的数据: w2c_matrix:0.995213,0.097735,-0.000735,-0.100000,-0.084959,0.861348,-0.500862,0.000000,-0.048319,0.498526,0.865529,0.400000,0.000000,0.000000,0.000000,1.000000
成功解析w2c矩阵:
tensor([[ 9.9521e-01,  9.7735e-02, -7.3500e-04, -1.0000e-01],
        [-8.4959e-02,  8.6135e-01, -5.0086e-01,  0.0000e+00],
        [-4.8319e-02,  4.9853e-01,  8.6553e-01,  4.0000e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
trying to : pre grouping...
buffers take : 89.70046043395996 ms    means3D 6.0062408447265625 ms    allocate 7.325649261474609 ms    temp_mem_buffer/sortouts_buffer 76.36666297912598 ms
Done! pre_grouping_ip takes : 53.122520446777344 ms
trying to : forward...
buffers take : 176.25904083251953 ms    sum acc_size 149.31392669677

KeyboardInterrupt: 

In [8]:
recv_sock.close()
send_sock.close()